In [19]:
from HodaDatasetReader import read_hoda_dataset


In [33]:
X_train, Y_train = read_hoda_dataset(dataset_path='./DigitDB/Train 60000.cdb',
                                images_height=32,
                                images_width=32,
                                one_hot=False,
                                reshape=True)
X_test, Y_test = read_hoda_dataset(dataset_path='./DigitDB/Test 20000.cdb',
                              images_height=32,
                              images_width=32,
                              one_hot=False,
                              reshape=True)
X_val, Y_val = read_hoda_dataset('./DigitDB/RemainingSamples.cdb',
                                             images_height=32,
                                             images_width=32,
                                             one_hot=False,
                                             reshape=True)

In [34]:
import tensorflow as tf 


In [35]:
# foe simplicity, we will scale the pixel intensities (range : 0-1)
# X_train,X_test,X_val=X_train / 255.,X_test/ 255.,X_val/ 255.

#### sequential model

In [37]:
# make the results reproducible
tf.random.set_seed(42)
model= tf.keras.Sequential([tf.keras.layers.Dense(300,activation='relu'),tf.keras.layers.Dense(150,activation='relu'),tf.keras.layers.Dense(10,activation='softmax')])


In [38]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [39]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

In [40]:
model.fit(X_train,Y_train,epochs=40,validation_data=(X_val,Y_val), callbacks=[callback])ڑ

Epoch 1/40
   7/1875 [..............................] - ETA: 16s - loss: 2.3122 - accuracy: 0.1295 

2023-02-19 15:09:55.093914: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1865/1875 [============================>.] - ETA: 0s - loss: 0.3326 - accuracy: 0.9143

2023-02-19 15:10:03.161808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 11s 6ms/step - loss: 0.3318 - accuracy: 0.9144 - val_loss: 0.2497 - val_accuracy: 0.9251
Epoch 2/40
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1424 - accuracy: 0.9608 - val_loss: 0.2022 - val_accuracy: 0.9403
Epoch 3/40
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1140 - accuracy: 0.9691 - val_loss: 0.1748 - val_accuracy: 0.9492
Epoch 4/40
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0958 - accuracy: 0.9737 - val_loss: 0.1569 - val_accuracy: 0.9539
Epoch 5/40
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0833 - accuracy: 0.9771 - val_loss: 0.1443 - val_accuracy: 0.9571
Epoch 6/40
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0732 - accuracy: 0.9802 - val_loss: 0.1283 - val_accuracy: 0.9618
Epoch 7/40
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0654 - accuracy: 0.9826 - val_loss: 0.1257 - val_accuracy: 0.9

#### functional

In [27]:
normaization_layer = tf.keras.layers.Normalization()
hidden_1=tf.keras.layers.Dense(300,activation='relu')
hidden_2=tf.keras.layers.Dense(250, activation='relu')
concat_layer= tf.keras.layers.Concatenate()
output_layer= tf.keras.layers.Dense(10,activation='softmax')

In [28]:
input_=tf.keras.layers.Input(shape=X_train.shape[1])
normalized = normaization_layer(input_)
hidden1=hidden_1(normalized)
hidden2=hidden_2(hidden1)
concat=concat_layer([hidden1,hidden2])
output=output_layer(concat)
model=tf.keras.Model(inputs=[input_],outputs=[output])


In [30]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [31]:
model.fit(X_train,Y_train,epochs=30,validation_data=(X_val,Y_val),callbacks=[callback])

Epoch 1/30


2023-02-19 15:00:41.713347: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-19 15:00:41.912637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1867/1875 [============================>.] - ETA: 0s - loss: 0.3232 - accuracy: 0.9187

2023-02-19 15:00:51.245921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 12s 6ms/step - loss: 0.3225 - accuracy: 0.9188 - val_loss: 0.2560 - val_accuracy: 0.9222
Epoch 2/30
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1456 - accuracy: 0.9602 - val_loss: 0.2081 - val_accuracy: 0.9389
Epoch 3/30
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1175 - accuracy: 0.9681 - val_loss: 0.1788 - val_accuracy: 0.9478
Epoch 4/30
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0994 - accuracy: 0.9734 - val_loss: 0.1623 - val_accuracy: 0.9530
Epoch 5/30
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0869 - accuracy: 0.9767 - val_loss: 0.1492 - val_accuracy: 0.9578
Epoch 6/30
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0767 - accuracy: 0.9796 - val_loss: 0.1325 - val_accuracy: 0.9620
Epoch 7/30
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0686 - accuracy: 0.9814 - val_loss: 0.1279 - val_accuracy: 0.9

همانط،ر که مشاهده می شود نتایج بسیار خوب شده است

In [32]:
model.evaluate(X_test,Y_test)

625/625 [==============================] - 3s 5ms/step - loss: 0.0935 - accuracy: 0.9728


[0.09351921081542969, 0.9727500677108765]

نتیجه داده های اعتبار سنجی با نتیجه بدست امده از داده های ازمایش به هم بسیار نزدیک می باشند

در این ازمایش هر ۲ ساحتار توانستن جواب یکسانی بدهند